<a href="https://colab.research.google.com/github/yallavaibhav/Comic-AudioBooks/blob/master/Final_data298.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install easyocr   

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import cv2 as cv
import pandas as pd
import easyocr
import matplotlib.pyplot as plt
import numpy as np
import easyocr
reader = easyocr.Reader(['en'],gpu=False)

Installation of Yolov5

In [3]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-134-g23c4923 Python-3.9.16 torch-2.0.0+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/107.7 GB disk)


**Panels**
- Croping the panels from the image using Yolov5 X

In [5]:
# Getting the cropped pics of the panels

# !python detect.py --save-txt --source  --weights 
!python /content/yolov5/detect.py --save-crop --save-txt --source /content/Venomnibus-v01-0006_jpg.rf.85d0de455b918337d0b360014e39b800.jpg --weights /content/best_yolov5X.pt

detect: weights=['/content/best_yolov5X.pt'], source=/content/Venomnibus-v01-0006_jpg.rf.85d0de455b918337d0b360014e39b800.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-134-g23c4923 Python-3.9.16 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
1 [[tensor(1.), tensor(11.), tensor(605.), tensor(185.)], [tensor(468.), tensor(185.), tensor(640.), tensor(377.)], [tensor(298.), tensor(186.), tensor(461.), tensor(375.)], [tensor(2.), tensor(181.), tensor(297.), tensor(640.)], [tensor(299.), tensor(379.), tensor(636.), tensor(635.)]]
5
2 [[tensor(1.), tensor(11.), tenso

In [6]:
# Storing the panels and sorting them in correct order
panels = []
for root,dir,files in os.walk(os.path.abspath("/content/yolov5/runs/detect/exp/crops/Panels/")):
  for file in files:
    panels.append((os.path.join(root, file)))
     

In [7]:
panels.sort()

In [8]:
panels

['/content/yolov5/runs/detect/exp/crops/Panels/kp.jpg',
 '/content/yolov5/runs/detect/exp/crops/Panels/kp2.jpg',
 '/content/yolov5/runs/detect/exp/crops/Panels/kp3.jpg',
 '/content/yolov5/runs/detect/exp/crops/Panels/kp4.jpg',
 '/content/yolov5/runs/detect/exp/crops/Panels/kp5.jpg']

**Characters**
- Saving the croped dialouge images with texts datausing Yolov5

In [14]:
for i in range(len(panels)):
  !python /content/yolov5/detect.py --save-crop --save-txt --source {panels[i]} --weights /content/best_speechballoons.pt

detect: weights=['/content/best_speechballoons.pt'], source=/content/yolov5/runs/detect/exp/crops/Panels/kp.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-134-g23c4923 Python-3.9.16 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 322 layers, 86213788 parameters, 0 gradients, 203.9 GFLOPs
1 [[tensor(444.), tensor(56.), tensor(577.), tensor(109.)]]
1
2 [[tensor(444.), tensor(56.), tensor(577.), tensor(109.)]]
[tensor(444.), tensor(56.), tensor(577.), tensor(109.)]
image 1/1 /content/yolov5/runs/detect/exp/crops/Panels/kp.jpg: 224x640 1 General_speech, 1356.5ms
Speed: 4.0ms pre-process, 1356.5ms inference, 2.0ms NMS per im

This function is for sorting the data

In [15]:
def sorting_speech_balloons(df,panel_number):
  sx = df.values.tolist()
  sx.sort(key=lambda x:x[2])
  #print(sx)
  x = []
  y = []
  k = 0
  for i in range(len(sx)):
    if sx[i][2] < sx[k][2]+0.1:
      x.append(sx[i])
      # print(x)
    else:
      x.sort(key=lambda x:x[1])
      for j in range(len(x)):
        y.append(x[j])
      # print(y)
      x = []
      # print(x)
      x.append(sx[i])
      k=i
  x.sort(key=lambda x:x[1])
  print("last",x)
  for p in range(len(x)):
    y.append(x[p])
  dff = pd.DataFrame(y)
  dff.columns = ["Class","X_center", "y_center", "height", "width"]
  dff.insert(0,'Panel_no',panel_number)
  return dff

Creating a dataframe from a single page (with different panels)

In [16]:
#Speech_balloon table with distance
speech_baloons_paths_main = []
temp = []
panel_number = 0
dataframe = pd.DataFrame(columns = ['Panel_no',	'Class',	'X_center',	'y_center',	'height',	'width'])
for i in range(2,len(panels)+2):
  for root,dir,files in os.walk(os.path.abspath("/content/yolov5/runs/detect/exp{0}/labels/".format(i))):
    for file in files:
      temp.append(os.path.join(root, file))
      print('sddddddddddddd',temp)
      # zz = '{fname}'.format(fname=temp)
for k in range(len(temp)):
  df = pd.read_csv(temp[k], header = None, sep=" ")
  panel_number += 1 
  dff = sorting_speech_balloons(df,panel_number)
      # print(dff)
  dataframe = dataframe.append(dff, ignore_index=True)

sddddddddddddd ['/content/yolov5/runs/detect/exp2/labels/kp.txt']
sddddddddddddd ['/content/yolov5/runs/detect/exp2/labels/kp.txt', '/content/yolov5/runs/detect/exp3/labels/kp2.txt']
sddddddddddddd ['/content/yolov5/runs/detect/exp2/labels/kp.txt', '/content/yolov5/runs/detect/exp3/labels/kp2.txt', '/content/yolov5/runs/detect/exp4/labels/kp3.txt']
sddddddddddddd ['/content/yolov5/runs/detect/exp2/labels/kp.txt', '/content/yolov5/runs/detect/exp3/labels/kp2.txt', '/content/yolov5/runs/detect/exp4/labels/kp3.txt', '/content/yolov5/runs/detect/exp5/labels/kp4.txt']
sddddddddddddd ['/content/yolov5/runs/detect/exp2/labels/kp.txt', '/content/yolov5/runs/detect/exp3/labels/kp2.txt', '/content/yolov5/runs/detect/exp4/labels/kp3.txt', '/content/yolov5/runs/detect/exp5/labels/kp4.txt', '/content/yolov5/runs/detect/exp6/labels/kp5.txt']
last [[0.0, 0.828734, 0.441176, 0.215909, 0.283422]]
last [[0.0, 0.715461, 0.136461, 0.569079, 0.230277]]
last [[0.0, 0.451705, 0.235149, 0.380682, 0.153465]]
l

<ipython-input-16-ea366b18d345>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)
<ipython-input-16-ea366b18d345>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)
<ipython-input-16-ea366b18d345>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)
<ipython-input-16-ea366b18d345>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)
<ipython-input-16-ea366b18d345>:17: FutureWarning: The frame.append method is deprecated and will be rem

This dataframe below is related to speech balloon locations only

In [17]:
dataframe

,Panel_no,Class,X_center,y_center,height,width
0,1,0.0,0.828734,0.441176,0.215909,0.283422
1,2,0.0,0.715461,0.136461,0.569079,0.230277
2,3,3.0,0.696023,0.099010,0.528409,0.118812
3,3,0.0,0.451705,0.235149,0.380682,0.153465
4,4,0.0,0.315642,0.197561,0.441341,0.258537
5,5,0.0,0.194286,0.079926,0.211429,0.100372
6,5,0.0,0.675714,0.172862,0.534286,0.301115


In [ ]:
#==================================================================================================================

The function provides the following pre-processing steps for text detection:
- Resizing 
- Thresholding
- Convex Hull
- Biggest Hull
- Gray Scaling
- Masking

Used EasyOCR for text detection

In [18]:
beta = []
def crop_convex_hull(img):
  img = cv.resize(img, (400, 350))
  Thresh_original = img.copy()
  Hull_biggestarea = img.copy()
  gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  _,binary = cv.threshold(gray,245,255,cv.THRESH_BINARY)
  contours1, hirechay1 = cv.findContours(binary,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
  con_thresh = cv.drawContours(Thresh_original,contours1, -1,(0,255,0),3) #-1 is to detect all contours, (255,255,255)=>green line, 3=> thickness
  #Biggest contours
  if len(contours1)>=1:  
    biggest_contours = max(contours1, key = cv.contourArea)
    hull_biggest = cv.convexHull(biggest_contours)
    cv.drawContours(Hull_biggestarea,[hull_biggest],0,(0,0,255),3)
    mask = np.zeros(gray.shape, np.uint8)
    cv.drawContours(mask, [hull_biggest], 0, 255, -1)
    result = cv.bitwise_and(img, img, mask=mask)
    # plt.imshow(result)

    results = reader.readtext(result)
  else:
    results = reader.readtext(img)
  return results
  # return result

def preprocess_text(images):
  gray = cv.cvtColor(images, cv.COLOR_BGR2GRAY)
  enhanced = cv.equalizeHist(gray)
  results = reader.readtext(enhanced)
  return results
def list_text(value):
  df = pd.DataFrame(value, columns=['bbox','text','confidence'])
  column_to_one_row = df['text'].tolist()
  One_sentence = ' '.join(column_to_one_row)
  sentence = One_sentence.lower()
  beta.append(sentence)
  return beta
# for i in speech_baloons_paths:
#   img = cv.imread(i)
#   res = list_text(crop_convex_hull(img))
#   print(beta)

In [19]:
speech_baloons_paths = []
temp = []
for i in range(2,len(panels)+2):
  for root,dir,files in os.walk(os.path.abspath("/content/yolov5/runs/detect/exp{0}/crops/".format(i))):
    for file in files:
      temp.append(os.path.join(root, file))
      # print(temp)
  temp.sort()
  speech_baloons_paths.append(temp)
  temp = []
for i in range(len(speech_baloons_paths)):
  for j in range(len(speech_baloons_paths[i])):
    print(speech_baloons_paths[i][j])
    img = cv.imread(speech_baloons_paths[i][j])
    res = list_text(crop_convex_hull(img))
print(beta)

/content/yolov5/runs/detect/exp2/crops/General_speech/kp.jpg
/content/yolov5/runs/detect/exp3/crops/General_speech/kp.jpg
/content/yolov5/runs/detect/exp4/crops/General_speech/kp.jpg
/content/yolov5/runs/detect/exp4/crops/narration speech/kp.jpg
/content/yolov5/runs/detect/exp5/crops/General_speech/kp.jpg
/content/yolov5/runs/detect/exp6/crops/General_speech/kp.jpg
/content/yolov5/runs/detect/exp6/crops/General_speech/kp2.jpg
['yolpe dt spicable : yoj and ail your kindi', 'flting onthe defenbeless vinging nothing but miseky: paini deauhhi', 'y make 03 sick!', 'wi mhatre tol f', 'gwlet 6 g8a kk', '5 ovlei', 'oli\'lldo but decay; in kivullis anp uption\' mingling th ythe filih filea"qf thf sewers of']


In [21]:
speech_baloons_paths

[['/content/yolov5/runs/detect/exp2/crops/General_speech/kp.jpg'],
 ['/content/yolov5/runs/detect/exp3/crops/General_speech/kp.jpg'],
 ['/content/yolov5/runs/detect/exp4/crops/General_speech/kp.jpg',
  '/content/yolov5/runs/detect/exp4/crops/narration speech/kp.jpg'],
 ['/content/yolov5/runs/detect/exp5/crops/General_speech/kp.jpg'],
 ['/content/yolov5/runs/detect/exp6/crops/General_speech/kp.jpg',
  '/content/yolov5/runs/detect/exp6/crops/General_speech/kp2.jpg']]

In [22]:
text_dataframe = pd.DataFrame(beta,columns=['Text'])
text_dataframe

,Text
0,yolpe dt spicable : yoj and ail your kindi
1,flting onthe defenbeless vinging nothing but m...
2,y make 03 sick!
3,wi mhatre tol f
4,gwlet 6 g8a kk
5,5 ovlei
6,oli'lldo but decay; in kivullis anp uption' mi...


In [23]:
Text_dataframe = dataframe.join(text_dataframe['Text'])

Joining both speech location data and its text into a single DataFrame

In [24]:
Text_dataframe

,Panel_no,Class,X_center,y_center,height,width,Text
0,1,0.0,0.828734,0.441176,0.215909,0.283422,yolpe dt spicable : yoj and ail your kindi
1,2,0.0,0.715461,0.136461,0.569079,0.230277,flting onthe defenbeless vinging nothing but m...
2,3,3.0,0.696023,0.099010,0.528409,0.118812,y make 03 sick!
3,3,0.0,0.451705,0.235149,0.380682,0.153465,wi mhatre tol f
4,4,0.0,0.315642,0.197561,0.441341,0.258537,gwlet 6 g8a kk
5,5,0.0,0.194286,0.079926,0.211429,0.100372,5 ovlei
6,5,0.0,0.675714,0.172862,0.534286,0.301115,oli'lldo but decay; in kivullis anp uption' mi...


**Characters**


In [27]:
for i in range(len(panels)):
  !python /content/yolov5/detect.py --save-txt --source {panels[i]} --weights /content/best_char.pt

detect: weights=['/content/best_char.pt'], source=/content/yolov5/runs/detect/exp/crops/Panels/kp.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-134-g23c4923 Python-3.9.16 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 322 layers, 86644444 parameters, 0 gradients, 205.3 GFLOPs
1 [[tensor(10.), tensor(51.), tensor(125.), tensor(173.)], [tensor(258.), tensor(72.), tensor(500.), tensor(178.)]]
2
2 [[tensor(10.), tensor(51.), tensor(125.), tensor(173.)], [tensor(258.), tensor(72.), tensor(500.), tensor(178.)]]
image 1/1 /content/yolov5/runs/detect/exp/crops/Panels/kp.jpg: 224x640 1 man, 1 venom, 1671.9ms
Speed: 4.2ms pre-

In [28]:
#Speech_balloon table with distance
# speech_baloons_paths_main = []
temp = []
panel_number = 0
dataframe = pd.DataFrame(columns = ['Panel_no',	'Class',	'X_center',	'y_center',	'height',	'width'])
for i in range(len(panels)+2,len(panels)+2+len(panels)):
  for root,dir,files in os.walk(os.path.abspath("/content/yolov5/runs/detect/exp{0}/labels/".format(i))):
    for file in files:
      temp.append(os.path.join(root, file))
      print('sddddddddddddd',temp)
      # zz = '{fname}'.format(fname=temp)
for k in range(len(temp)):
  df = pd.read_csv(temp[k], header = None, sep=" ")
  panel_number += 1 
  dff = sorting_speech_balloons(df,panel_number)
      # print(dff)
  dataframe = dataframe.append(dff, ignore_index=True)

sddddddddddddd ['/content/yolov5/runs/detect/exp7/labels/kp.txt']
sddddddddddddd ['/content/yolov5/runs/detect/exp7/labels/kp.txt', '/content/yolov5/runs/detect/exp8/labels/kp2.txt']
sddddddddddddd ['/content/yolov5/runs/detect/exp7/labels/kp.txt', '/content/yolov5/runs/detect/exp8/labels/kp2.txt', '/content/yolov5/runs/detect/exp9/labels/kp3.txt']
sddddddddddddd ['/content/yolov5/runs/detect/exp7/labels/kp.txt', '/content/yolov5/runs/detect/exp8/labels/kp2.txt', '/content/yolov5/runs/detect/exp9/labels/kp3.txt', '/content/yolov5/runs/detect/exp11/labels/kp5.txt']
last [[65.0, 0.109578, 0.59893, 0.186688, 0.652406], [69.0, 0.61526, 0.668449, 0.392857, 0.566845]]
last [[70.0, 0.271382, 0.852878, 0.542763, 0.294243]]
last [[47.0, 0.457386, 0.509901, 0.823864, 0.871287]]
last [[20.0, 0.201429, 0.739777, 0.265714, 0.394052]]


<ipython-input-28-3f82c25fc022>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)
<ipython-input-28-3f82c25fc022>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)
<ipython-input-28-3f82c25fc022>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)
<ipython-input-28-3f82c25fc022>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(dff, ignore_index=True)


In [29]:
Character_table = dataframe

In [30]:
Character_table

,Panel_no,Class,X_center,y_center,height,width
0,1,65.0,0.109578,0.598930,0.186688,0.652406
1,1,69.0,0.615260,0.668449,0.392857,0.566845
2,2,65.0,0.231908,0.297441,0.457237,0.390192
3,2,69.0,0.620066,0.466951,0.575658,0.477612
4,2,70.0,0.271382,0.852878,0.542763,0.294243
5,3,47.0,0.457386,0.509901,0.823864,0.871287
6,4,69.0,0.424286,0.500000,0.637143,0.873606
7,4,70.0,0.844286,0.501859,0.180000,0.342007
8,4,20.0,0.201429,0.739777,0.265714,0.394052


In [31]:
Text_dataframe

,Panel_no,Class,X_center,y_center,height,width,Text
0,1,0.0,0.828734,0.441176,0.215909,0.283422,yolpe dt spicable : yoj and ail your kindi
1,2,0.0,0.715461,0.136461,0.569079,0.230277,flting onthe defenbeless vinging nothing but m...
2,3,3.0,0.696023,0.099010,0.528409,0.118812,y make 03 sick!
3,3,0.0,0.451705,0.235149,0.380682,0.153465,wi mhatre tol f
4,4,0.0,0.315642,0.197561,0.441341,0.258537,gwlet 6 g8a kk
5,5,0.0,0.194286,0.079926,0.211429,0.100372,5 ovlei
6,5,0.0,0.675714,0.172862,0.534286,0.301115,oli'lldo but decay; in kivullis anp uption' mi...


Join the tables

Speech Cloning:
1. Collect data for classes
2. Train for Sythesiser, encoder
3. if else code
4. Storage

Storage:
1. Store the speech
2. Store the panels data


Moviepy